Micah Gale

19 Sep 25

# NE 536: HomeWork 1


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


## Problem 1.1 Plasma Parameters

Given:

* $T_{e^-} = 1 keV$
* $n = 5 \times 10^{18} \frac{1}{m^3}$
* $|\vec{B}| = 1 T$

In [2]:
# store inputs as constants
TEMP_E = 1_000 # [eV]
DENSITY = 5e18 # [m^-3]
FIELD_ST = 1.0 # T

Calculate:
* Debye Length
* Plasma frequency
* electron collision frequency

### Debye Length

$\lambda_d = \sqrt{
\frac{\epsilon_0 T_e}{ne^2}}$

We have density, $n$, and electron temperature, $T_{e^-}$, so we just need the constants: vacuum permittivity, $\epsilon_0$, and the elementary charge, $e$.

I used:

* $\epsilon_0 = 8.854 \times 10^{-12}\frac{F}{m}$ [1]
* $e = 1.602\times 10^{-19} C$ [2]

[1] “Vacuum permittivity,” Wikipedia. July 20, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Vacuum_permittivity&oldid=1301610312

[2] “Elementary charge,” Wikipedia. June 03, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Elementary_charge&oldid=1293751741



In [3]:
EPSILON_0 = 8.854e-12 #[F/m]
ELEM_CHARGE = 1.602e-19 #[C]

In [4]:
temp_e_j = TEMP_E * ELEM_CHARGE # convert to Joules
debye_l = np.sqrt((EPSILON_0 * temp_e_j) / (DENSITY * ELEM_CHARGE ** 2))
print(f"Debye Length for plasma at electron temperature {TEMP_E / 1000:.1f} keV, and a density of " 
      f"{DENSITY:.1e} m^-3 is:\n {debye_l* 1000:.3f} mm")

Debye Length for plasma at electron temperature 1.0 keV, and a density of 5.0e+18 m^-3 is:
 0.105 mm


### Plasma Frequency

The plasma frequency, $\omega_p$ is given by:

$\omega_p = \sqrt{\frac{ne^2}{m_e\epsilon_0}}$

We just need the mass of an electron, $m_e$ now:

* $m_e = 9.10938 \times 10^{-31} kg$  [3]

[3] “Electron mass,” Wikipedia. May 26, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Electron_mass&oldid=1292442002


In [5]:
MASS_E = 9.10938e-31 #[kg]

plasma_ang_freq = np.sqrt((DENSITY * ELEM_CHARGE**2)/ (MASS_E * EPSILON_0))
plasma_freq = plasma_ang_freq / (2.0 * np.pi)
print(f"Frequency for plasma at electron temperater at a density of " 
      f"{DENSITY:.1e} m^-3 is: {plasma_freq/1e9:.3f} GHz")

Frequency for plasma at electron temperater at a density of 5.0e+18 m^-3 is: 20.075 GHz


### Electron Collision Frequency


The electron collision frequency is the inverse of the collison time (Freidberg).

$\nu_c = \frac{1}{\tau_c}  = \frac{n\lambda}{T_e^{3/2} 3.44\times 10^6}$. 

This requires $\lambda$, the Coloumb logarithm which is defined for electron-electron collison [7]:

$\lambda_{ee} = 23.5 - \ln\left(\frac{n^{1/2}}{T^{5/4}}\right) - \left(10^{-5} + \frac{(\ln(T_e) -2)^2}{16}\right)^{1/2}$


[7] A. Berensyak, “2023 NRL Plasma Formulary,” Naval Research Laboratory, Washington, D.C., 2023. [Online]. Available: https://www.nrl.navy.mil/News-Media/Publications/nrl-plasma-formulary/


In [6]:
col_log = 23.5 - np.log(np.sqrt(DENSITY)/ np.power(TEMP_E, 5/4))
col_log -= np.sqrt(1e-5 + (np.log(TEMP_E) -2)**2/16)

colision_freq = (DENSITY * col_log) / (np.power(TEMP_E, 3/2)*3.44e6)

In [7]:
print(f"The Electron Collision frequency for a plasma at a Temperature of: {TEMP_E/1e3:.2f} keV and "
      f"a density of: {DENSITY:.1e} m^-3 is {colision_freq/1e6:.2f} MHz")

The Electron Collision frequency for a plasma at a Temperature of: 1.00 keV and a density of: 5.0e+18 m^-3 is 431.13 MHz


## Problem 1.2 Thermal Velocities

Calculate thermal velocity (mean speed) for electron and hydrogen ion (proton) at:
* $T = 1keV$
* $T = 10 keV$


Will need the mass of a proton:

* $m_p = 1.6726\times 10^{-27} kg$ [4]

[4] “Proton - Wikipedia.” Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/wiki/Proton

### Will any be realtivistic?

In [8]:
TEMPERATURES = [1e3, 10e3] # [eV]
MASS_P = 1.6726e-27 # [kg]

In [9]:
# set up the dataframe
buffer = []
for temp in TEMPERATURES:
    for part in {"electron", "hydrogen"}:
        buffer.append({"particle": part, "temperature [eV]": temp})
df = pd.DataFrame(buffer)
df["mass [kg]"] = df["particle"].apply(lambda p: MASS_P if p == "hydrogen" else MASS_E)

The Maxwell-Boltzmann equation can give us:

* $<v> = \frac{2}{\sqrt{pi}} v_p$
* $v_p = \sqrt{\frac{2k_BT}{m}}$

Where:

* $<v>$ is the average speed
* $v_p$ is the most probable speed
* $k_B$ is the Boltzmann constant [5], which is:


* $k_B = 1.3806\times 10^{-23}\frac{J}{K}$ [6]

[5] “Maxwell–Boltzmann distribution,” Wikipedia. Aug. 19, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Maxwell%E2%80%93Boltzmann_distribution&oldid=1306808676#Typical_speeds

[6] “Boltzmann constant,” Wikipedia. July 11, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Boltzmann_constant&oldid=1300011910#Role_in_the_equipartition_of_energy



In [10]:
BOLTZMANN = 1.3806e-23 #[j/k]
def get_mean_speed(row):
    temp, mass = row[["temperature [eV]", "mass [kg]"]]
    v_p = np.sqrt((2*BOLTZMANN * temp)/mass)
    return 2/np.sqrt(np.pi)*v_p

df["mean velocity [m/s]"] = df.apply(get_mean_speed, axis=1)

### Is it relativistic?
For this we are saying $v > 0.1c$ is relativistic,
with $c$ being the speed of light:

* $c= 299 792 458 \frac{m}{s}$ [7]

[7] “Speed of light,” Wikipedia. Sept. 15, 2025. Accessed: Sept. 16, 2025. [Online]. Available: https://en.wikipedia.org/w/index.php?title=Speed_of_light&oldid=1311507394


In [11]:
SPEED_LIGHT = 299_792_458 #[m/s]

df["frac c"] = df["mean velocity [m/s]"] / SPEED_LIGHT
df["relativistic"] = df["frac c"] > 0.1
df

,particle,temperature [eV],mass [kg],mean velocity [m/s],frac c,relativistic
0,hydrogen,1000.0,1.672600e-27,4584.668569,0.000015,False
1,electron,1000.0,9.109380e-31,196453.414744,0.000655,False
2,hydrogen,10000.0,1.672600e-27,14497.994994,0.000048,False
3,electron,10000.0,9.109380e-31,621240.244707,0.002072,False


Therefore, no, none of these particles at these energies are relativistic. The closest is the electron at 10 keV at 0.2% $c$.

# Problem 1.3

Plot the new END/B-VIII.1 D(T,n) cross sections!

![the plot](Updated_xs_plot.png)

The only update was a slight cross section decrease at about 8 MeV, but also includes almost 1 MeV of more data.

# Problem 1.4 Orbit Following

Drift in a toroidal magnetic field is ultimately caused by a magnetic force, and is proportional to said force.
This has to follow:

$$ \vec{F} = q\vec{v} \times \vec{B}$$

Obviously for a particle $q$ cannot change so only $\vec{B}$ will change. 
For the magnetic field strength is proportional the applied force, $\vec{F}$, and therefore the drift.

For the particle energy, of course, $E=1/2mv^2$, and so velocity is proportional to: $\sqrt{E}$. 
Therefore the drift in the torodial field is proportional to $\sqrt{E}$.